### Create External Delta Lake Table for Taxi Zones

#### Based on:

https://github.com/MrPowers/delta-examples/blob/master/notebooks/pyspark/create-table-delta-lake.ipynb

In [0]:
import pandas as pd
import io
import os
import requests

In [0]:
url = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv'

web_content = requests.get(url).content
pandas_dataframe = pd.read_csv(io.StringIO(web_content.decode('utf-8')))

In [0]:
display(pandas_dataframe)

In [0]:
# access_key = dbutils.secrets.get(scope = 'aws', key = 'access_key')
# secret_key = dbutils.secrets.get(scope = 'aws', key = 'secret_key')

access_key = os.getenv('AWS_ACCESS_KEY')
secret_key = os.getenv('AWS_SECRET_KEY')

BUCKET_NAME = 'dmitov-nyc-taxi'
DIRECTORY = 'delta-lake-taxi-zones'

spark_dataframe = spark.createDataFrame(pandas_dataframe)

spark_dataframe.write.mode('overwrite').format('delta').save(
  's3a://{}:{}@{}/{}'.format(
      access_key,
      secret_key,
      BUCKET_NAME,
      DIRECTORY
  )
)

In [0]:
control_spark_dataframe = spark.read.format('delta').load(
  's3a://{}:{}@{}/{}'.format(
      access_key,
      secret_key,
      BUCKET_NAME,
      DIRECTORY
  )
)

display(control_spark_dataframe.sort('LocationID'))